In [1]:
import pandas as pd 
import numpy as np 
import re
import json 

In [2]:
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

What is the state breakdown of the gendered phrases?

In [3]:
masculine_themed_wording = [
    "competitive",
    "dominate",
    "leader",
    "rock star",
    "rockstar",
    "guru",
    "ninja",
    "hacker",
    "superhero",
    "prove themselves",
    "analyze",
    "determine",
    "crush it",
    "world class",
    "superior",
    "ambitious",
    "aggressive",
    "leader"
]

In [4]:
data_set = pd.read_csv("../data/dice/us-technology-jobs-on-dicecom/dice_com-job_us_sample.csv")
data_set = data_set.get(['jobtitle','jobdescription','joblocation_address'])
states = pd.read_csv("states.csv")

In [55]:
data_set['State'] = data_set['joblocation_address'].apply(lambda x: pd.Series(str(x).split(", ")[-1]))

In [56]:
descriptionDict = createTokenizedJobDescriptionDict(data_set)

In [6]:
ps = PorterStemmer()

In [15]:
def numOfOcurrences(tokenizedPhrase, tokenizedDescription): 
#     counter = 0
    # if its a phrase just look for the exact phrase in the description otherwise look at the stem of the words
#     if len(word_tokenize(phrase)) > 1: 
#         regex = re.compile(phrase)
#         return len(re.findall(regex, description))
    
#     desc = word_tokenize(description) 
#     desc = list(map(lambda x: ps.stem(x), desc))
#     return desc.count(stem)

    counter = 0
    i = len(tokenizedPhrase)
    for x in range(len(tokenizedDescription)-i + 1):
        if tokenizedDescription[x:x+i] == tokenizedPhrase:
            counter += 1; 
    return counter

In [8]:
def stem_tokens(s): 
    ps = PorterStemmer()
    tokens = word_tokenize(s)
    stem = list(map(lambda x : ps.stem(x), tokens))
    
    return stem 

In [59]:
def indicesOfJobsWithPhrase(phrase, df, descriptionDict): 
    indices = []
    for index, row in df.iterrows(): 
        tokenizedPhrase = stem_tokens(phrase)
#         tokenizedDesc = stem_tokens(row.get("jobdescription"))
        if numOfOcurrences(tokenizedPhrase, descriptionDict[index]) > 0: 
            indices.append(index)
    return indices

In [46]:
def createTokenizedJobDescriptionDict(df): 
    descriptions = {}
    for index, row in df.iterrows(): 
        descriptions[index] = stem_tokens(row.get("jobdescription"))
    return descriptions

In [95]:
def computeStateInfo(state, full_df,descriptions): 
    stateInfo = {}
    stateInfo['region'] = state
    stateInfo['word_count'] = {}
    state_df = full_df.where(full_df['State'] == state).dropna()
    biasedIndices = []; 
    
    for word in masculine_themed_wording: 
        
        indices = indicesOfJobsWithPhrase(word, state_df, descriptions)
        stateInfo['word_count'][word] = len(indices)
        biasedIndices += indices
    
    topWords = [item[0] for item in reversed(sorted(stateInfo['word_count'].items(), key = lambda kv: kv[1]))][:5]
    
    stateInfo['numberOfJobsInDataSet'] = len(state_df)
    stateInfo['numberOfBiasedJobs'] = len(set(biasedIndices))
    try: 
        stateInfo['percentageOfBiasedJobs'] = float(stateInfo['numberOfBiasedJobs']/ stateInfo['numberOfJobsInDataSet']) * 100
    except ZeroDivisionError: 
        stateInfo['percentageOfBiasedJobs'] = 'NA'
        #     stateInfo['biasedIndices'] = list(set(biasedIndices))
                                                                                                               
    stateInfo['questionFour'] = topWords
    del(stateInfo['word_count'])
    return stateInfo
    

In [93]:
def exportInfo(DF): 
    data = [computeStateInfo(state, DF,descriptionDict) for state in states['Code']]
    stateData = {"data": data}; 
    with open('stateData.json', 'w') as outfile:
        json.dump(stateData, outfile)
    
        

In [96]:
import time

startTime = time.time()
exportInfo(data_set)
runtime = time.time() - startTime
runtime

124.61947178840637